In [1]:
# 데이터 불러오기

In [190]:
import pandas as pd
import numpy as np

In [191]:
df = pd.read_csv('./car.data', header = 0)
df_new = pd.read_csv('./car.data', header = 0)

In [192]:
df.columns

Index(['buying', 'maint', 'doors', 'persons', 'lug_boot', 'safety', 'class'], dtype='object')

In [193]:
# 독립변수와 종속변수 모두 범주형 데이터이므로, 모델링하기 위해 원-핫 인코딩으로 더미변수로 변환.
data_dummies = pd.get_dummies(df)
display(data_dummies.head())

,buying_high,buying_low,buying_med,buying_vhigh,maint_high,maint_low,maint_med,maint_vhigh,doors_2,doors_3,...,lug_boot_big,lug_boot_med,lug_boot_small,safety_high,safety_low,safety_med,class_acc,class_good,class_unacc,class_vgood
0,0,0,0,1,0,0,0,1,1,0,...,0,0,1,0,1,0,0,0,1,0
1,0,0,0,1,0,0,0,1,1,0,...,0,0,1,0,0,1,0,0,1,0
2,0,0,0,1,0,0,0,1,1,0,...,0,0,1,1,0,0,0,0,1,0
3,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,1,0,0,0,1,0
4,0,0,0,1,0,0,0,1,1,0,...,0,1,0,0,0,1,0,0,1,0


In [194]:
# 더미변수화 된 데이터 독립변수와 종속변수로 나누기. 
x_features = data_dummies.loc[:, 'buying_high':'safety_med']
X = x_features.values
y = df['class']

In [195]:
# 1) 결정트리

In [196]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

# train, test 셋 분할
X_train, X_test, y_train, y_test = train_test_split(
X, y, stratify=y, random_state=0)

# 결정트리 학습
tree = DecisionTreeClassifier(random_state=0)
tree.fit(X_train, y_train)

# 결정트리의 테스트 셋 정확도는 0.96
# 오버피팅.

print("train set score is {:.3f}".format(tree.score(X_train, y_train)))
print("test set score is {:.3f}".format(tree.score(X_test, y_test)))


# 예측값과 실제값 상위 5개 비교 => 모두 일치.

print()
print(tree.predict(X_test)[:5].tolist())
print(y_test[:5].tolist())

# 위에서부터 10개 데이터는 모두 일치함.

dt_pred = tree.predict(X_test)[:10]
test_1 = y_test[:10]
np.all(dt_pred == test_1)

train set score is 1.000
test set score is 0.954

['unacc', 'unacc', 'unacc', 'unacc', 'unacc']
['unacc', 'unacc', 'unacc', 'unacc', 'unacc']


True

In [197]:
# 랜덤포레스트, 그레이디언트 부스팅 회귀 트리

In [198]:
# 랜덤 포레스트
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100, random_state = 0)
forest.fit(X_train, y_train)
print("train set score is {:.3f}".format(forest.score(X_train, y_train)))
print("test set score is {:.3f}".format(forest.score(X_test, y_test)))

print()
print(forest.predict(X_test)[:5].tolist())
print(y_test[:5].tolist())

# 10개 데이터 일치 확인
rf_pred = forest.predict(X_test)[:10]
np.all(rf_pred == test_1)

train set score is 1.000
test set score is 0.933

['acc', 'unacc', 'unacc', 'unacc', 'unacc']
['unacc', 'unacc', 'unacc', 'unacc', 'unacc']


False

In [201]:
# 그레이디언트 부스팅 회귀 트리

from sklearn.ensemble import GradientBoostingClassifier

gbrt = GradientBoostingClassifier(random_state = 0)
gbrt.fit(X_train, y_train)

print("train set score is {:.3f}".format(gbrt.score(X_train, y_train)))
print("test set score is {:.3f}".format(gbrt.score(X_test, y_test)))

print()
print(gbrt.predict(X_test)[:10].tolist())
print(y_test[:10].tolist())


# 10개 데이터 일치 확인
gbrt_pred = gbrt.predict(X_test)[:10]
np.all(gbrt_pred == test_1)

train set score is 0.992
test set score is 0.979

['unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'unacc']
['unacc', 'unacc', 'unacc', 'unacc', 'unacc', 'acc', 'unacc', 'unacc', 'unacc', 'unacc']


False

In [202]:
# df 자체에서 y값을 1, 2, 3, 4로 바꾸기.(수학적 모델들의 성능 향상 위해/)

In [203]:
df_new.replace('unacc', '0', inplace = True)
df_new.replace('acc', '1', inplace = True)
df_new.replace('good', '2', inplace = True)
df_new.replace('vgood', '3', inplace = True)

In [204]:
df_new
y_1d = df_new.loc[:, "class"]
y_1d

0       0
1       0
2       0
3       0
4       0
       ..
1723    2
1724    3
1725    0
1726    2
1727    3
Name: class, Length: 1728, dtype: object

In [205]:
# train, test 셋 분할
X_train, X_test, y_train, y_test = train_test_split(X, y_1d, stratify=y, random_state=0)

In [180]:
# 커널 서포트 벡터 분류기

from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', C=1, gamma = 0.1)
svm.fit(X_train, y_train)

print("train set score is {:.3f}".format(svm.score(X_train, y_train)))
print("test set score is {:.3f}".format(svm.score(X_test, y_test)))

print()
print(svm.predict(X_test)[:10].tolist())
print(y_test[:10].tolist())

# 10개 데이터 일치 확인    # 하나 틀림.
sv_pred = svm.predict(X_test)[:10]
np.all(sv_pred == test_2)

train set score is 0.965
test set score is 0.944

['0', '1', '0', '0', '0', '0', '1', '0', '3', '0']
['0', '2', '0', '0', '0', '0', '1', '0', '3', '0']


False

In [181]:
# 커널 서포트 벡터 분류기(매개변수 조정)

from sklearn.svm import SVC
svm = SVC(kernel = 'rbf', C=10, gamma = 0.1)
svm.fit(X_train, y_train)

print("train set score is {:.3f}".format(svm.score(X_train, y_train)))
print("test set score is {:.3f}".format(svm.score(X_test, y_test)))

print()
print(svm.predict(X_test)[:5].tolist())
print(y_test[:5].tolist())

# 10개 데이터 일치 확인
sv_pred = svm.predict(X_test)[:10]
np.all(sv_pred == test_2)

train set score is 1.000
test set score is 0.998

['0', '2', '0', '0', '0']
['0', '2', '0', '0', '0']


True

In [182]:
# 로지스틱 회귀

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(solver='liblinear', multi_class='auto', C=100.0, random_state=1)
lr.fit(X_train, y_train)
print("train set score is {:.3f}".format(lr.score(X_train, y_train)))
print("test set score is {:.3f}".format(lr.score(X_test, y_test)))

print()
print(lr.predict(X_test)[:5].tolist())
print(y_test[:5].tolist())

# 10개 데이터 일치 확인
lr_pred = lr.predict(X_test)[:10]
np.all(lr_pred == test_2)

train set score is 0.910
test set score is 0.905

['0', '1', '0', '0', '0']
['0', '2', '0', '0', '0']


False